In [1]:
import pandas as pd
import datetime

In [2]:
df1 = pd.read_csv("./satellite_breakup(from PDF).csv")
df1[["LAUNCH DATE", "BREAKUP DATE"]] = df1[["LAUNCH DATE", "BREAKUP DATE"]].apply(pd.to_datetime)
df1["launch_designator"] = df1["INTERNATIONAL DESIGNATOR"].str.slice(stop=8)
df1 = df1[['SATELLITE NAME', 'INTERNATIONAL DESIGNATOR', 'US SATELLITE NUMBER', 'LAUNCH DATE', 'BREAKUP DATE', 'DEBRIS CATALOGED', 'DEBRIS ON-ORBIT', 'launch_designator']]

df2 = pd.read_csv("../satcat_all.csv")
df2[["DEBUT", "DECAY", "LAUNCH"]] = df2[["DEBUT", "DECAY", "LAUNCH"]].apply(pd.to_datetime)
df2["launch_designator"] = df2.INTLDES.str.slice(stop=8)
df2 = df2[['INTLDES', 'NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'DEBUT', 'LAUNCH', 'DECAY', 'launch_designator']]

In [3]:
def fix_date(x):
    if x.year > 2056:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

# dates < 1969 are parsed as 20xx, fix it
df1['BREAKUP DATE'] = df1['BREAKUP DATE'].apply(fix_date)
df1['LAUNCH DATE'] = df1['LAUNCH DATE'].apply(fix_date)

US SATELLITE NUMBER == NORAD_CAT_ID

Check if PDF data includes anything that doesn't exist in SATCAT

NOPE! That's great

In [4]:
df1[~df1["US SATELLITE NUMBER"].isin(df2.NORAD_CAT_ID.values)]
# 0 rows = everything matches up

,SATELLITE NAME,INTERNATIONAL DESIGNATOR,US SATELLITE NUMBER,LAUNCH DATE,BREAKUP DATE,DEBRIS CATALOGED,DEBRIS ON-ORBIT,launch_designator


In [5]:
combined = df2.merge(df1[["launch_designator","BREAKUP DATE"]], how="left", left_on="launch_designator", right_on="launch_designator")
combined[(combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].notnull()]
# combined[(combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].isnull()]

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,DEBUT,LAUNCH,DECAY,launch_designator,BREAKUP DATE
129,1961-015AA,140,DEBRIS,THOR ABLESTAR DEB,2002-01-13 00:00:00,1961-06-29,1962-09-30,1961-015,1961-06-29
130,1961-015AB,141,DEBRIS,THOR ABLESTAR DEB,2004-10-07 11:32:23,1961-06-29,NaT,1961-015,1961-06-29
131,1961-015AC,142,DEBRIS,THOR ABLESTAR DEB,2002-01-13 00:00:00,1961-06-29,1967-12-25,1961-015,1961-06-29
132,1961-015AD,143,DEBRIS,THOR ABLESTAR DEB,2002-01-13 00:00:00,1961-06-29,1962-06-16,1961-015,1961-06-29
133,1961-015AE,144,DEBRIS,THOR ABLESTAR DEB,2004-10-07 11:32:23,1961-06-29,NaT,1961-015,1961-06-29
...,...,...,...,...,...,...,...,...,...
45210,2017-086H,43094,DEBRIS,SL-23 DEB,2017-12-28 16:33:09,2017-12-26,2018-01-14,2017-086,2018-02-12
45211,2017-086J,43219,DEBRIS,FREGAT DEB,2018-02-24 16:39:41,2017-12-26,2018-05-11,2017-086,2018-02-12
45212,2017-086K,43220,DEBRIS,FREGAT DEB,2018-02-24 16:39:41,2017-12-26,2018-04-17,2017-086,2018-02-12
45213,2017-086L,43221,DEBRIS,FREGAT DEB,2018-02-24 16:39:41,2017-12-26,2018-02-24,2017-086,2018-02-12


In [6]:
df2.loc[(df2.OBJECT_TYPE!="DEBRIS"),"exist_date"] = df2[["DEBUT", "LAUNCH"]].min(axis=1)

df2[df2.OBJECT_TYPE=="DEBRIS"] all have exist_date which are NaN now.  Let's try to fill it in from the PDF data

As seen below, there are 13 incidents where the same launch had 2 documented breakup dates. Unfortunately we cannot assign the proper breakup date for debris generated from these events. However, there aren't too many of these debris, we may be able to get away with assigning 1 of the breakup dates to all debris with the same starting launch designator.

In [7]:
t = df1.launch_designator.str.slice(stop=8).value_counts()
t[t>=2]

2008-067    2
1989-001    2
2002-037    2
1990-045    2
1984-011    2
1990-110    2
1987-079    2
1992-047    2
2010-007    2
1999-057    2
1986-069    2
1988-085    2
1991-025    2
Name: launch_designator, dtype: int64

In [8]:
display(df1[df1["INTERNATIONAL DESIGNATOR"]=="1961-015C"])
print("Documented in satcat:",len(df2[df2.INTLDES.str.startswith("1961-015C") & (df2.OBJECT_TYPE == "DEBRIS")]))
# can see from this example that the PDF file DEBRIS CATALOGED and DEBRIS ON-ORBIT is more than SATCAT

,SATELLITE NAME,INTERNATIONAL DESIGNATOR,US SATELLITE NUMBER,LAUNCH DATE,BREAKUP DATE,DEBRIS CATALOGED,DEBRIS ON-ORBIT,launch_designator
0,TRANSIT 4A R/B,1961-015C,118,1961-06-29,1961-06-29,296,172.0,1961-015


Documented in satcat: 24
